In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install opencv-contrib-python
!pip install diffusers transformers accelerate

In [3]:
!git clone https://github.com/huggingface/diffusers

fatal: destination path 'diffusers' already exists and is not an empty directory.


In [4]:
%cd diffusers/
!pip install -r examples/controlnet/requirements.txt

/content/diffusers


In [5]:
!pip install omegaconf

In [12]:
!python ./scripts/convert_original_controlnet_to_diffusers.py \
 --checkpoint_path /content/drive/MyDrive/CS245\ Project/Saved_weights/default_epoch=039_step=016000.ckpt  \
 --original_config_file /content/drive/MyDrive/CS245\ Project/Code/ControlNet-Trainer/models/cldm_v15.yaml \
 --dump_path /content/drive/MyDrive/CS245\ Project/Code/ControlNet-Trainer/trained-40-epochs

2023-12-08 11:46:54.410655: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-08 11:46:54.410715: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-08 11:46:54.410756: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-08 11:46:57.736369: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [13]:
#@title Test data
import json

source_path = "/content/drive/MyDrive/CS245 Project/Test/"
test_data = []
with open(source_path + "caption-test.json", 'rt') as f:
    for line in f:
        test_data.append(json.loads(line))

test_data[0]

{'source': 'source/00008270_015.png',
 'target': 'target/00008270_015.png',
 'prompt': 'Chest x-ray of a patient with no diseases'}

In [14]:
import os
import cv2
import pandas as pd
from PIL import Image
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel, UniPCMultistepScheduler
import torch
import numpy as np
from diffusers.utils import load_image

controlnet = ControlNetModel.from_pretrained(
        "/content/drive/MyDrive/CS245 Project/Code/ControlNet-Trainer/trained-40-epochs",
        torch_dtype=torch.float16
    )

In [15]:
pipe = StableDiffusionControlNetPipeline.from_pretrained(
        "runwayml/stable-diffusion-v1-5", controlnet=controlnet, safety_checker=None, torch_dtype=torch.float16
    )

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

You have disabled the safety checker for <class 'diffusers.pipelines.controlnet.pipeline_controlnet.StableDiffusionControlNetPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


In [16]:
pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)
pipe.enable_model_cpu_offload()

In [ ]:
for data in test_data:
  image_filename = os.path.basename(data["source"])
  if os.path.isfile(source_path + data["source"]):
    image = load_image(source_path + data["source"])

    # low_threshold = 100
    # high_threshold = 200

    # image = cv2.Canny(image, low_threshold, high_threshold)
    # image = image[:, :, None]
    # image = np.concatenate([image, image, image], axis=2)
    # image = Image.fromarray(image)

    # controlnet = ControlNetModel.from_pretrained(
    #     "lllyasviel/sd-controlnet-canny", torch_dtype=torch.float16
    # )

    # pipe = StableDiffusionControlNetPipeline.from_pretrained(
    #     "runwayml/stable-diffusion-v1-5", controlnet=controlnet, safety_checker=None, torch_dtype=torch.float16
    # )

    # pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)

    # # Remove if you do not have xformers installed
    # # see https://huggingface.co/docs/diffusers/v0.13.0/en/optimization/xformers#installing-xformers
    # # for installation instructions
    # # pipe.enable_xformers_memory_efficient_attention()

    # pipe.enable_model_cpu_offload()

    image = pipe(data["prompt"], image, num_inference_steps=20).images[0]
    print("prompt:", data["prompt"])

    image.save('/content/drive/MyDrive/CS245 Project/ControlNet_Finetune_TestOutput_40/' + image_filename)

  0%|          | 0/20 [00:00<?, ?it/s]

prompt: Chest x-ray of a patient with no diseases


  0%|          | 0/20 [00:00<?, ?it/s]

prompt: Chest x-ray of a patient with diseases


  0%|          | 0/20 [00:00<?, ?it/s]

...

  0%|          | 0/20 [00:00<?, ?it/s]

prompt: Chest x-ray of a patient with diseases


  0%|          | 0/20 [00:00<?, ?it/s]

prompt: Chest x-ray of a patient with no diseases


  0%|          | 0/20 [00:00<?, ?it/s]